In [1]:
pip install --upgrade tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import os
import glob as gb
import cv2
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
np.random.seed(42)


In [3]:
data_dir = "Datasets"
IMG_HEIGHT = 224
IMG_WIDTH  = 224
channels = 3

In [4]:
for folder in os.listdir(data_dir):
  files = gb.glob(pathname=str(data_dir + '/' + folder + '/*.JPG'))
  print(f'Total data found {len(files)} in folder {folder}')

Total data found 48 in folder Bacteria
Total data found 50 in folder Fungi
Total data found 46 in folder Nematodes
Total data found 39 in folder Normal
Total data found 49 in folder Virus


In [5]:
import random
import matplotlib.image as mpimg
class_names = ['Normal','Fungi','Bacteria','Nematodes','Virus']
image_data = []

image_labels = []

for i in class_names:

  path= data_dir + '/' + str(i)
  images = os.listdir(path)


  for img in images:

    try:

      image = cv2.imread(path + '/' + img)

      image_fromarray =Image.fromarray(image, 'RGB')

      resize_image =image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
      image_data.append(np.array(resize_image))       

      image_labels.append(i)

    except:

      print("Error in " + img)


#Changing the list to numpy array image_data = np.array(image_data)
image_data = np.array(image_data)

image_labels= np.array(image_labels)

print(image_data.shape, image_labels.shape)

(232, 224, 224, 3) (232,)


In [6]:
shuffle_indexes = np.arange(image_data.shape[0])

np.random.shuffle(shuffle_indexes)

image_data = image_data[shuffle_indexes]

image_labels = image_labels [shuffle_indexes]



In [7]:
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelEncoder

labelencoder=LabelEncoder()


image_labels=to_categorical(labelencoder.fit_transform(image_labels))

In [8]:
image_labels

array([[0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.]])

In [9]:
X_train, X_val, y_train, y_val =train_test_split(image_data, image_labels, test_size=0.3, random_state=42, shuffle=True)


X_train = X_train/255

X_val = X_val/255

print("X_train.shape", X_train.shape)

print("X_valid.shape", X_val.shape)

print("y_train.shape", y_train.shape)

print("y_valid.shape", y_val.shape)

X_train.shape (162, 224, 224, 3)
X_valid.shape (70, 224, 224, 3)
y_train.shape (162, 5)
y_valid.shape (70, 5)


In [10]:
model = keras.models.Sequential([

  keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation ='relu', input_shape=(IMG_HEIGHT,IMG_WIDTH,channels)),

  keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'),

  keras.layers.MaxPool2D(pool_size=(2, 2)),

#keras.layers.BatchNormalization(axis=-1),

  keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'),

  keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu'),

  keras.layers.MaxPool2D(pool_size=(2, 2)),

#keras.layers.BatchNormalization(axis=-1),

  keras.layers.Flatten(),

  keras.layers.Dense(512, activation='relu'),

#keras.layers.BatchNormalization(),

  keras.layers.Dropout(rate=0.5),

  keras.layers.Dense(5, activation='softmax')
])

C:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Learning rate and decay settings
lr = 0.001
epochs = 30

# Define the exponential decay for learning rate
lr_schedule = ExponentialDecay(
    initial_learning_rate=lr,
    decay_steps=epochs // 2,  # or any appropriate step size
    decay_rate=0.5,  # Decay rate
    staircase=True)

# Use the Adam optimizer with the learning rate schedule
opt = Adam(learning_rate=lr_schedule)

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


In [ ]:
aug = ImageDataGenerator(

rotation_range=10,

zoom_range=0.15,

width_shift_range=0.1,

height_shift_range=0.1,

shear_range=0.15,

horizontal_flip=False,

vertical_flip=False,

fill_mode="nearest")

history = model.fit(aug.flow(X_train, y_train, batch_size=32),epochs=epochs,validation_data=(X_val,y_val))

C:\Users\ACER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 316s 50s/step - accuracy: 0.1444 - loss: 3.7169 - val_accuracy: 0.2714 - val_loss: 1.7042
Epoch 2/30
5/6 ━━━━━━━━━━━━━━━━━━━━ 44s 44s/step - accuracy: 0.2617 - loss: 1.6185 